In [13]:
pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=de9ee714ca4a64bfda947804caea718f533de580c2a61e6cce6f05056614af4d
  Stored in directory: C:\Users\예해주\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [41]:
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 

# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 

code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()


,name,code
0,AK홀딩스,006840
1,DSR,155660
2,GS,078930
3,GS글로벌,001250
4,HDC현대산업개발,294870


In [42]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code[1:])

    print("요청 URL = {}".format(url)) 
    return url 

# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠' 
url = get_url(item_name, code_df) 

# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 

# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 100): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    
# df.dropna()를 이용해 결측값 있는 행 제거 
df = df.dropna() 

# 상위 5개 데이터 확인하기 
df.head()


요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
1,2020.04.10,12250.0,800.0,12600.0,13000.0,11350.0,9453105.0
2,2020.04.09,13050.0,50.0,13250.0,13500.0,13000.0,2686952.0
3,2020.04.08,13100.0,550.0,13550.0,13650.0,13100.0,2724264.0
4,2020.04.07,13650.0,500.0,13400.0,14150.0,13100.0,8519795.0
5,2020.04.06,13150.0,100.0,13050.0,13450.0,12900.0,3550426.0


In [43]:
# 한글로 된 컬럼명을 영어로 바꿔줌
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', 
                         '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})

# 데이터의 타입을 int형으로 바꿔줌 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df['date'] = pd.to_datetime(df['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df = df.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume
1471,2016-12-06,12850,650,13500,15050,12400,13725568
1231,2016-12-06,12850,650,13500,15050,12400,13725568
1246,2016-12-06,12850,650,13500,15050,12400,13725568
1261,2016-12-06,12850,650,13500,15050,12400,13725568
1276,2016-12-06,12850,650,13500,15050,12400,13725568


In [44]:
# 필요한 모듈 import 하기 
import plotly.offline as offline 
import plotly.graph_objs as go 

# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True) 

trace = go.Scatter( x=df.date, y=df.close, name=item_name) 

data = [trace]

# data = [celltrion] 
layout = dict( 
    title='{}의 종가(close) Time Series'.format(item_name), 
    xaxis=dict( 
        rangeselector=dict( 
            buttons=list([ 
                dict(count=1, 
                     label='1m', 
                     step='month', 
                     stepmode='backward'), 
                dict(count=3, 
                     label='3m', 
                     step='month', 
                     stepmode='backward'), 
                dict(count=6, 
                     label='6m', 
                     step='month', 
                     stepmode='backward'), 
                dict(step='all') 
            ]) 
        ), 
        rangeslider=dict(), 
        type='date' 
    ) 
)

fig = go.Figure(data=data, layout=layout) 
offline.iplot(fig)

In [45]:
def get_macd(df, short=12, long=26, t=9): 
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌 
    df = pd.DataFrame(df) 
    
    # MACD 관련 수식 
    ma_12 = df.close.ewm(span=12).mean() # 단기(12) EMA(지수이동평균) 
    ma_26 = df.close.ewm(span=26).mean() # 장기(26) EMA 
    macd = ma_12 - ma_26 # MACD 
    macds = macd.ewm(span=9).mean() # Signal 
    macdo = macd - macds # Oscillator 
    
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna() 
    
    return df

In [46]:
df = get_macd(df) 

# 상위 5개 데이터 확인 
df.head()


,date,close,diff,open,high,low,volume,macd,macds,macdo
1471,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0
1231,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0
1246,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0
1261,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0
1276,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0


In [57]:
from plotly import tools 
import plotly.offline as offline 
import plotly.graph_objs as go 

macd = go.Scatter( x=df.date, y=df['macd'], name="MACD") 
signal = go.Scatter( x=df.date, y=df['macds'], name="Signal") 
oscillator = go.Bar( x=df.date, y=df['macdo'], name="oscillator") 
trade_volume = go.Bar( x=df.date, y=df['volume'], name="volume") 

data = [macd, signal, oscillator] 

# data = [celltrion] 
layout = go.Layout(title='{} MACD 그래프'.format(item_name)) 

fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True) 

for trace in data: 
    fig.append_trace(trace, 1,1) 
    
fig.append_trace(trade_volume, 2,1) 
# fig = go.Figure(data=data, layout=layout) 

offline.iplot(fig)


In [48]:
# 일자(n,m,t)에 따른 Stochastic(KDJ)의 값을 구하기 위해 함수형태로 만듬 
def get_stochastic(df, n=15, m=5, t=3):
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌
    df = pd.DataFrame(df)
    
    # n일중 최고가
    ndays_high = df.high.rolling(window=n, min_periods=1).max()
    # n일중 최저가
    ndays_low = df.low.rolling(window=n, min_periods=1).min()
 
    # Fast%K 계산
    kdj_k = ((df.close - ndays_low) / (ndays_high - ndays_low))*100
    # Fast%D (=Slow%K) 계산
    kdj_d = kdj_k.ewm(span=m).mean()
    # Slow%D 계산
    kdj_j = kdj_d.ewm(span=t).mean()
 
    # dataframe에 컬럼 추가
    df = df.assign(kdj_k=kdj_k, kdj_d=kdj_d, kdj_j=kdj_j).dropna()
    
    return df


In [49]:
df = get_stochastic(df)
df.head()

,date,close,diff,open,high,low,volume,macd,macds,macdo,kdj_k,kdj_d,kdj_j
1471,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0,16.981132,16.981132,16.981132
1231,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0,16.981132,16.981132,16.981132
1246,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0,16.981132,16.981132,16.981132
1261,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0,16.981132,16.981132,16.981132
1276,2016-12-06,12850,650,13500,15050,12400,13725568,0.0,0.0,0.0,16.981132,16.981132,16.981132


In [56]:
import plotly.offline as offline 
import plotly.graph_objs as go
from plotly import tools
 
# jupyter notebook 에서 출력 
offline.init_notebook_mode(connected=True)
 
kdj_k = go.Scatter(
                x=df.date,
                y=df['kdj_k'],
                name="Fast%K")
 
kdj_d = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Fast%D")
 
kdj_d2 = go.Scatter(
                x=df.date,
                y=df['kdj_d'],
                name="Slow%K")
 
kdj_j = go.Scatter(
                x=df.date,
                y=df['kdj_j'],
                name="Slow%D")
 
 
trade_volume = go.Bar(
                x=df.date,
                y=df['volume'],
                name="volume")
 
 
# data = [kdj_k, kdj_d]
data1 = [kdj_d2, kdj_j]
data2 = [trade_volume]
 
# data = [celltrion]
# layout = go.Layout(yaxis=dict(
#         autotick=False,
#         ticks='outside',
#         tick0=0,
#         dtick=10,
#         ticklen=8,
#         tickwidth=4,
#         tickcolor='#000'
#     ))
 
fig = tools.make_subplots(rows=2, cols=1, shared_xaxes=True)
 
for trace in data1:
    fig.append_trace(trace, 1,1)
 
for trace in data2:
    fig.append_trace(trace, 2,1)
# fig = go.Figure(data=data, layout=layout)
 
offline.iplot(fig)

In [52]:
ma5 = df['close'].rolling(window=5).mean()
ma20 = df['close'].rolling(window=20).mean()
ma60 = df['close'].rolling(window=60).mean()
ma120 = df['close'].rolling(window=120).mean()
ma5.tail()

5    13170.0
4    13250.0
3    13250.0
2    13200.0
1    13040.0
Name: close, dtype: float64

In [54]:
df = df.assign(ma5=ma5, ma20=ma20, ma60=ma60, ma120=ma120).dropna()

In [ ]:
df.to_csv('./naver_stock.csv')

In [60]:
writer = pd.ExcelWriter("test_excel.xlsx")
df.to_excel(writer, sheet_name='Day')
writer.save()

In [ ]:
pip install ta-lib